# 신경망 언어모형 실습: Tokenizer

## 데이터 받기

imdb.zip을 다운로드 받는다.

In [1]:
import requests

res = requests.get('https://github.com/euphoris/datasets/raw/master/imdb.zip')

with open('../data/imdb.zip', 'wb') as f:
    f.write(res.content)

In [1]:
import pandas as pd
df = pd.read_csv('../data/imdb.zip')

In [2]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


## 토큰화

텐서플로의 Tokenizer를 사용한다.

In [3]:
import tensorflow as tf

상위 2000개의 단어까지만 번호로 변환하고 나머지는 unk로 취급한다.

In [4]:
tk = tf.keras.preprocessing.text.Tokenizer(num_words=2000, oov_token='<unk>')

단어에 번호를 붙인다.

In [5]:
tk.fit_on_texts(df['review'])

In [6]:
tk.word_index

{'<unk>': 1,
 'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'is': 6,
 'this': 7,
 'i': 8,
 'it': 9,
 'to': 10,
 'in': 11,
 'was': 12,
 'movie': 13,
 'film': 14,
 'that': 15,
 'for': 16,
 'as': 17,
 'but': 18,
 'with': 19,
 'one': 20,
 'on': 21,
 'you': 22,
 'are': 23,
 'not': 24,
 'bad': 25,
 "it's": 26,
 'very': 27,
 'all': 28,
 'just': 29,
 'so': 30,
 'good': 31,
 'at': 32,
 'an': 33,
 'be': 34,
 'there': 35,
 'about': 36,
 'have': 37,
 'by': 38,
 'like': 39,
 'from': 40,
 'if': 41,
 'acting': 42,
 'time': 43,
 'out': 44,
 'his': 45,
 'or': 46,
 'really': 47,
 'great': 48,
 'even': 49,
 'he': 50,
 'who': 51,
 'were': 52,
 'has': 53,
 'see': 54,
 'my': 55,
 'characters': 56,
 'well': 57,
 'most': 58,
 'how': 59,
 'more': 60,
 'no': 61,
 'only': 62,
 'when': 63,
 'ever': 64,
 '10': 65,
 'movies': 66,
 'plot': 67,
 'story': 68,
 'made': 69,
 'some': 70,
 'they': 71,
 'best': 72,
 'because': 73,
 'your': 74,
 'can': 75,
 'also': 76,
 "don't": 77,
 'films': 78,
 'than': 79,
 'its': 80,
 'scrip

In [7]:
tk.word_index['good']

31

In [8]:
tk.index_word[31]

'good'

토크나이저를 저장한다.

In [8]:
import joblib
joblib.dump(tk, '../data/tokenizer.pkl')

['tokenizer.pkl']

# 신경망 언어모형 실습: 전처리

## 준비

데이터를 연다.

In [9]:
import pandas as pd
df = pd.read_csv('../data/imdb.zip')

토크나이저를 불러온다.

In [10]:
import joblib
tk = joblib.load('../data/tokenizer.pkl')

## 언어 모형에 맞게 데이터 정리

텍스트를 번호로 바꿔주는 작업

In [11]:
seqs = tk.texts_to_sequences(df['review'])

In [12]:
seqs[0]

[4, 27, 27, 27, 287, 407, 1217, 13, 36, 4, 1218, 1219, 408, 142]

In [13]:
tk.index_word[4]

'a'

In [14]:
tk.index_word[27]

'very'

In [18]:
seq = seqs[0]

In [19]:
for i in range(0, len(seq) - 4):
        print((seq[i:i+4], seq[i+4]))

([4, 27, 27, 27], 287)
([27, 27, 27, 287], 407)
([27, 27, 287, 407], 1217)
([27, 287, 407, 1217], 13)
([287, 407, 1217, 13], 36)
([407, 1217, 13, 36], 4)
([1217, 13, 36, 4], 1218)
([13, 36, 4, 1218], 1219)
([36, 4, 1218, 1219], 408)
([4, 1218, 1219, 408], 142)


In [20]:
data = []
for seq in seqs:
    for i in range(0, len(seq) - 4):
        data.append((seq[i:i+4], seq[i+4]))

In [25]:
data[0]

([4, 27, 27, 27], 287)

데이터를 섞어준다.

In [26]:
import random

In [27]:
random.shuffle(data)

In [28]:
data[0]

([904, 53, 485, 69], 4)

x와 y로 데이터를 나눈다.

In [29]:
import numpy as np

xs = np.array([x for x, y in data])
ys = np.array([y for x, y in data])

In [30]:
xs

array([[ 904,   53,  485,   69],
       [  53,   10,  382,  508],
       [  24,   10,  781,  524],
       ...,
       [  31,  975,  303,   66],
       [ 887,    4,   57, 1106],
       [ 109,   19,    2,  369]])

저장한다.

In [31]:
joblib.dump((xs, ys), '../data/lm-data.pkl')

['lm-data.pkl']

# 신경망 언어모형 실습: 학습

## 준비

준비된 데이터를 불러온다.

In [32]:
import joblib
tk = joblib.load('../data/tokenizer.pkl')
xs, ys = joblib.load('../data/lm-data.pkl')

## 언어 모형

언어모형에 들어갈 임베딩 레이어를 만든다.

In [34]:
import tensorflow as tf

In [35]:
tk.num_words

2000

단어 번호가 1번부터 붙으므로 0번까지 포함하면 총 단어 수에 1을 더해야 한다.

In [37]:
NUM_WORD = tk.num_words + 1

In [38]:
lm = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=NUM_WORD, output_dim=8),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(NUM_WORD),
])

In [43]:
emb1 = tf.keras.layers.Embedding(
    input_dim=NUM_WORD,
    output_dim=8,
)

언어 모형을 만든다.

In [49]:
lm = tf.keras.Sequential([
    emb1,
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(NUM_WORD)
])

모형 요약을 확인한다.

In [39]:
lm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 8)           16008     
_________________________________________________________________
global_average_pooling1d (Gl (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 2001)              18009     
Total params: 34,089
Trainable params: 34,089
Non-trainable params: 0
_________________________________________________________________


첫번째 행의 두번째 None은 n-gram의 n을 의미

3번째 행의 72는 (8+1)*8을 의미

모형을 학습시킨다.

(from_logits=True) 소프트맥스 생략시 손실을 계산할 때 소프트맥스를 함

In [40]:
lm.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [41]:
lm.fit(xs, ys, epochs=1)

331/331 [==============================] - 3s 4ms/step - loss: 7.3064 - accuracy: 0.0703


모형을 저장한다.

In [42]:
lm.save('lm.krs')

INFO:tensorflow:Assets written to: lm.krs\assets


## 단어 임베딩

단어 임베딩을 확인한다.

In [28]:
e = emb1.embeddings.numpy()

In [29]:
e.shape

(2001, 8)

단어 임베딩을 임베딩 레이어의 가중치와 동일하다.

In [30]:
import numpy as np

w = emb1.get_weights()[0]
np.array_equal(e, w)

True

임베딩을 저장한다.

In [59]:
np.savez('../data/word-emb.npz', emb=e)

# GlobalAveragePooling1D

In [60]:
import tensorflow as tf
import numpy as np

GlobalAveragePooling1D는 1번 인덱스를 기준으로 평균을 구한다. 예를 들기 위해 다음과 같은 행렬이 있다고 하자

In [61]:
x = np.array([[[1, 2, 3], [3, 6, 9]]], dtype='float32')
x

array([[[1., 2., 3.],
        [3., 6., 9.]]], dtype=float32)

In [62]:
x.shape

(1, 2, 3)

이 행렬을 GlobalAveragePooling1D 레이어에 통과시키면 다음과 같이 된다.

In [63]:
avg = tf.keras.layers.GlobalAveragePooling1D()

In [64]:
y = avg(x).numpy()
y

array([[2., 4., 6.]], dtype=float32)

In [65]:
y.shape

(1, 3)

# 신경망 언어모형 실습: 다음 토큰의 확률 예측

## 준비

준비된 데이터를 불러온다.

In [66]:
import joblib
tk = joblib.load('../data/tokenizer.pkl')
xs, ys = joblib.load('../data/lm-data.pkl')

학습된 모형을 불러온다.

In [67]:
import tensorflow as tf

In [68]:
lm = tf.keras.models.load_model('../data/lm.krs')

## 다음에 나올 단어의 확률 예측

In [77]:
x = xs[0:1]
y = ys[0]

x의 4단어를 확인한다.

In [78]:
[tk.index_word[i] for i in x[0]]

['schrader', 'has', 'indeed', 'made']

모형에 넣는다.

In [79]:
import numpy as np

In [80]:
logit = lm.predict(x.astype('float32'))

In [81]:
logit

array([[-2.6672814,  2.567117 ,  2.3327217, ..., -2.6826978, -2.6719108,
        -2.7002132]], dtype=float32)

In [82]:
logit.shape

(1, 2001)

소프트맥스 함수를 적용하여 확률로 바꾼다.

In [83]:
p = tf.nn.softmax(logit).numpy()
p

array([[0.00011476, 0.0215307 , 0.01703184, ..., 0.000113  , 0.00011423,
        0.00011104]], dtype=float32)

여기에서 실제로 나온 단어를 확인한다.

In [84]:
tk.index_word[y]

'a'

해당 단어의 확률을 본다.

In [85]:
p[0, y]

0.011836766

확률이 가장 높은 단어를 알아본다.

In [86]:
i = p.argmax()
i

1

In [87]:
p[0, i]

0.021530699

In [88]:
tk.index_word[i]

'<unk>'

# 신경망 언어모형 실습: 전이 학습

## 실습 준비

IMDB 리뷰 데이터를 받는다.

데이터를 연다.

In [89]:
import pandas as pd
df = pd.read_csv('https://github.com/euphoris/datasets/raw/master/imdb.zip')

토크나이저를 불러들인다.

In [90]:
import joblib
tk = joblib.load('../data/tokenizer.pkl')

텍스트를 토큰의 번호 시퀀스로 바꾼다.

In [91]:
seqs = tk.texts_to_sequences(df['review'])

In [92]:
seqs[0]

[4, 27, 27, 27, 287, 407, 1217, 13, 36, 4, 1218, 1219, 408, 142]

In [93]:
seqs[1]

[24, 522, 51, 12, 60, 409, 2, 736, 56, 46, 2, 337, 1220, 288, 5, 737, 738, 44]

시퀀스마다 길이가 모두 다르므로 앞에 0을 채워(padding) 길이를 맞춰준다.

In [94]:
import tensorflow as tf

In [95]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs)

In [96]:
pads[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    4,   27,   27,   27,  287,  407, 1217,
         13,   36,    4, 1218, 1219,  408,  142])

## 단어 임베딩 불러오기

학습된 단어 임베딩을 불러온다.

In [97]:
import numpy as np

z = np.load('../data/word-emb.npz')
e = z['emb']

In [98]:
e

array([[ 0.00246316,  0.00992881, -0.04943324, ...,  0.01150197,
         0.00170362,  0.01206745],
       [ 0.04054418,  0.01958554,  0.03012717, ...,  0.04117012,
         0.0204666 , -0.04284757],
       [-0.00349189, -0.03143039, -0.03849567, ..., -0.04686766,
         0.00854615,  0.03951854],
       ...,
       [-0.02762789,  0.02321875,  0.03799022, ..., -0.00223788,
         0.02448225, -0.00046618],
       [ 0.03322499,  0.04585669, -0.02113488, ..., -0.02608277,
        -0.01219501,  0.04522601],
       [ 0.03026653, -0.04895736, -0.02045484, ..., -0.00444834,
         0.0185187 ,  0.04666949]], dtype=float32)

## 감성 분석

감성 분석 모형에 들어갈 임베딩 레이어를 만든다. 언어 모형에서 학습된 가중치로 초기화한다.

In [99]:
emb2 = tf.keras.layers.Embedding(
    input_dim=tk.num_words + 1,
    output_dim=8,
    embeddings_initializer=tf.keras.initializers.Constant(e) # 언어모형에서 학습됐던 임베딩을 기본값(시작값)으로 넣어줌
)


감성 분석 모형을 만든다.

In [100]:
model = tf.keras.Sequential([
    emb2,
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # 0 아니면 1 숫자하나만 예측
])

모형 요약을 확인한다.

In [101]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 8)           16008     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 8)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 16,089
Trainable params: 16,089
Non-trainable params: 0
_________________________________________________________________


모형을 설정한다.

In [102]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

학습시킨다.

In [103]:
y = df['sentiment'].values

In [104]:
model.fit(pads, y)

32/32 [==============================] - 1s 5ms/step - loss: 0.6938 - accuracy: 0.4877
